# PRE - Use of postgres in local 
DAOA
21/08/2025

* I am trying to get to connect to my local postgres db, i think i broke my raspi by setting up the ssh, i need to fix it.
* I have setup a local postgres db and created a historical table
* I want to now load the information


In [6]:
import pandas as pd
import numpy as np
import psycopg2
import yaml
from sqlalchemy import create_engine, inspect

In [4]:
# import yaml with credentials
with open("creds.yaml", "r") as file:
    creds = yaml.safe_load(file)["postgres"]
# Create connection
engine = create_engine(
    f"postgresql+psycopg2://{creds['user']}:{creds['password']}@{creds['host']}:{creds['port']}/{creds['database']}"
)


In [10]:
# Check what is available

inspector = inspect(engine)

tables = inspector.get_table_names(schema="public")

print(tables)

['test', 'historical_prices']


In [13]:
# Check which columns are there
columns = inspector.get_columns("historical_prices", schema="public")

for col in columns:
    print(col["name"], col["type"])

id INTEGER
stock VARCHAR(20)
date TIMESTAMP
open_price NUMERIC(12, 4)
high_price NUMERIC(12, 4)
low_price NUMERIC(12, 4)
close_price NUMERIC(12, 4)
volume BIGINT
daily_change NUMERIC(8, 4)
daily_range NUMERIC(8, 4)
daily_return NUMERIC(8, 4)
volatility_20d NUMERIC(8, 4)
created_at TIMESTAMP


In [28]:
## Get historical data: 


## Setup: 
import pandas as pd
import numpy as np
import yfinance as yf

## Declare portfolio: 
big_tech = {
    "AAPL": "Apple Inc.",
    "MSFT": "Microsoft Corporation", 
    "GOOGL": "Alphabet Inc. (Google) - Class A",
    "GOOG": "Alphabet Inc. (Google) - Class C",
    "TSLA": "Tesla Inc.",
    "AMZN": "Amazon.com Inc.",
    "META": "Meta Platforms Inc. (Facebook)",
    "NVDA": "NVIDIA Corporation", 
    "UBER": "Uber Technologies",
    "NFLX": "Netflix Inc.",
    "ORCL": "Oracle Corporation",
    "CRM": "Salesforce Inc.",
    "ADBE": "Adobe Inc.",
    "INTC": "Intel Corporation",
    "AMD": "Advanced Micro Devices",
    "CSCO": "Cisco Systems Inc.",
    "IBM": "International Business Machines",
    "QCOM": "QUALCOMM Incorporated",
    "TXN": "Texas Instruments",
    "TSM": "Taiwan Semiconductor",     # World's largest chip manufacturer
    "ARM": "Arm Holdings",            # CPU architecture (recent IPO)
}

indexes = {
    "^GSPC": "S&P 500",              # Market benchmark
    "^VIX": "Volatility Index",       # Fear gauge
    "^IXIC": "NASDAQ Composite",     # Tech-heavy index
    "^MXX": "Mexico's top stock"
}

major_banks = {
    "JPM": "JPMorgan Chase & Co.",                  # Largest US bank
    "BAC": "Bank of America Corp.",                 # Consumer banking giant
    "C": "Citigroup Inc.",                          # Global banking
    "GS": "Goldman Sachs Group",                    # Investment banking
    "MS": "Morgan Stanley",                         # Wealth management
    "COF": "Capital One Financial",                 # Credit cards/digital banking
    "BBVA": "Global BBVA", 
    "GFNORTEO.MX": "Banorte", 
    "DBK.DE": "Deutsche Bank"
}

fintech_stocks = {
    "PYPL": "PayPal Holdings",                      # You already have this
    "MA": "Mastercard Inc.",                        # Payment networks
    "V": "Visa Inc.",                               # Payment networks
    "AXP": "American Express Co."                  # Premium cards/payments
}

major_mexican = {
    "AMX":    "América Móvil",                       # Largest telecom in LatAm
    "BIMBOA.MX":  "Grupo Bimbo",                         # Global bakery leader
    "CEMEXCPO.MX": "Cemex",                              # Cement & construction
    "GMEXICOB.MX": "Grupo México",                       # Mining & infrastructure
    "WALMEX.MX":  "Walmart de México",                   # Retail giant
    #"KOFL.MX":    "Coca-Cola FEMSA",                     # Coke bottler
    "KIMBERA.MX": "Kimberly-Clark de México",            # Consumer products
    "ALSEA.MX":   "Alsea",                               # Restaurants operator
    "GCARSOA1.MX": "Grupo Carso",                        # Carlos Slim’s conglomerate
}

## Declare df:
stock_df = pd.DataFrame()

## BIG TECH
for symbol in big_tech.keys():
    print(f"Fetching {symbol}...")
    ticker = yf.Ticker(symbol)
    hist = ticker.history(period='max', interval='1d')
    
    # Create DataFrame with date as a column
    tmp_df = pd.DataFrame({
        'Stock': symbol,
        'Date': hist.index,  # ← This was missing!
        'Open': hist['Open'],
        'High': hist['High'],    # ← Add this for better analysis
        'Low': hist['Low'],      # ← Add this too
        'Close': hist['Close'],
        'Volume': hist['Volume'], 
        'Type': 'Big Tech'
    })
    
    stock_df = pd.concat([stock_df, tmp_df], ignore_index=True)
print('Done')

## INDICES
for symbol in indexes.keys():
    print(f"Fetching {symbol}...")
    ticker = yf.Ticker(symbol)
    hist = ticker.history(period='max', interval='1d')
    
    # Create DataFrame with date as a column
    tmp_df = pd.DataFrame({
        'Stock': symbol,
        'Date': hist.index,  # ← This was missing!
        'Open': hist['Open'],
        'High': hist['High'],    # ← Add this for better analysis
        'Low': hist['Low'],      # ← Add this too
        'Close': hist['Close'],
        'Volume': hist['Volume'], 
        'Type': 'Indexes'
    })
    
    stock_df = pd.concat([stock_df, tmp_df], ignore_index=True)
print('Done')

## BANKING
for symbol in major_banks.keys():
    print(f"Fetching {symbol}...")
    ticker = yf.Ticker(symbol)
    hist = ticker.history(period='max', interval='1d')
    
    # Create DataFrame with date as a column
    tmp_df = pd.DataFrame({
        'Stock': symbol,
        'Date': hist.index,  # ← This was missing!
        'Open': hist['Open'],
        'High': hist['High'],    # ← Add this for better analysis
        'Low': hist['Low'],      # ← Add this too
        'Close': hist['Close'],
        'Volume': hist['Volume'], 
        'Type': 'Major Banks'
    })
    
    stock_df = pd.concat([stock_df, tmp_df], ignore_index=True)
print('Done')


## FINTECH
for symbol in fintech_stocks.keys():
    print(f"Fetching {symbol}...")
    ticker = yf.Ticker(symbol)
    hist = ticker.history(period='max', interval='1d')
    
    # Create DataFrame with date as a column
    tmp_df = pd.DataFrame({
        'Stock': symbol,
        'Date': hist.index,  # ← This was missing!
        'Open': hist['Open'],
        'High': hist['High'],    # ← Add this for better analysis
        'Low': hist['Low'],      # ← Add this too
        'Close': hist['Close'],
        'Volume': hist['Volume'], 
        'Type': 'Fintech'
    })
    
    stock_df = pd.concat([stock_df, tmp_df], ignore_index=True)
print('Done')


## MEXICAN STOCKS
for symbol in major_mexican.keys():
    print(f"Fetching {symbol}...")
    ticker = yf.Ticker(symbol)
    hist = ticker.history(period='max', interval='1d')
    
    # Create DataFrame with date as a column
    tmp_df = pd.DataFrame({
        'Stock': symbol,
        'Date': hist.index,  # ← This was missing!
        'Open': hist['Open'],
        'High': hist['High'],    # ← Add this for better analysis
        'Low': hist['Low'],      # ← Add this too
        'Close': hist['Close'],
        'Volume': hist['Volume'], 
        'Type': 'Major Mexican'
    })
    
    stock_df = pd.concat([stock_df, tmp_df], ignore_index=True)
print('Done')

# Add calculations
stock_df['Daily_Change'] = ((stock_df['Close'] - stock_df['Open']) / stock_df['Open']) * 100
stock_df['Daily_Range'] = ((stock_df['High'] - stock_df['Low']) / stock_df['Open']) * 100
stock_df['Daily_Return'] = stock_df.groupby('Stock')['Close'].pct_change() * 100
stock_df['Volatility_20d'] = stock_df.groupby('Stock')['Daily_Return'].transform(
    lambda x: x.rolling(window=20, min_periods=5).std()
)

print(stock_df.head())


Fetching AAPL...
Fetching MSFT...
Fetching GOOGL...
Fetching GOOG...
Fetching TSLA...
Fetching AMZN...
Fetching META...
Fetching NVDA...
Fetching UBER...
Fetching NFLX...
Fetching ORCL...
Fetching CRM...
Fetching ADBE...
Fetching INTC...
Fetching AMD...
Fetching CSCO...
Fetching IBM...
Fetching QCOM...
Fetching TXN...
Fetching TSM...
Fetching ARM...
Done
Fetching ^GSPC...
Fetching ^VIX...
Fetching ^IXIC...
Fetching ^MXX...
Done
Fetching JPM...
Fetching BAC...
Fetching C...
Fetching GS...
Fetching MS...
Fetching COF...
Fetching BBVA...
Fetching GFNORTEO.MX...
Fetching DBK.DE...
Done
Fetching PYPL...
Fetching MA...
Fetching V...
Fetching AXP...
Done
Fetching AMX...
Fetching BIMBOA.MX...
Fetching CEMEXCPO.MX...
Fetching GMEXICOB.MX...
Fetching WALMEX.MX...
Fetching KIMBERA.MX...
Fetching ALSEA.MX...
Fetching GCARSOA1.MX...
Done
  Stock                       Date      Open      High       Low     Close  \
0  AAPL  1980-12-12 00:00:00-05:00  0.098485  0.098913  0.098485  0.098485   
1  AAPL

In [29]:
stock_df.columns

Index(['Stock', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Type',
       'Daily_Change', 'Daily_Range', 'Daily_Return', 'Volatility_20d'],
      dtype='object')

In [30]:
stock_df.isnull().sum()

Stock               0
Date                0
Open                0
High                0
Low                 0
Close               0
Volume              0
Type                0
Daily_Change        0
Daily_Range         0
Daily_Return       46
Volatility_20d    230
dtype: int64

In [31]:
# prepare data

stock_df.rename(columns={
    "Stock": "stock",
    "Date": "date",
    "Open": "open_price",
    "High": "high_price",
    "Low": "low_price",
    "Close": "close_price",
    "Volume": "volume",
    "Type": "type",
    "Daily_Change": "daily_change",
    "Daily_Range": "daily_range",
    "Daily_Return": "daily_return",
    "Volatility_20d": "volatility_20d"
}, inplace=True)

# add date created
from datetime import datetime 
stock_df["created_at"] = datetime.now()

In [32]:
## TIME TO UPLOAD

# Upload DataFrame
stock_df.to_sql(
    "historical_prices",
    con=engine,
    schema="public",
    if_exists="append",  # append to existing table
    index=False
)
print('done')

DataError: (psycopg2.errors.NumericValueOutOfRange) numeric field overflow
DETAIL:  A field with precision 8, scale 4 cannot hold an infinite value.

[SQL: INSERT INTO public.historical_prices (stock, date, open_price, high_price, low_price, close_price, volume, type, daily_change, daily_range, daily_return, volatility_20d, created_at) VALUES (%(stock__0)s, %(date__0)s, %(open_price__0)s, %(high_price__ ... 261407 characters truncated ... e__999)s, %(daily_range__999)s, %(daily_return__999)s, %(volatility_20d__999)s, %(created_at__999)s)]
[parameters: {'created_at__0': datetime.datetime(2025, 8, 21, 13, 54, 24, 823001), 'close_price__0': 8.982097625732422, 'volatility_20d__0': 1.3580425400707614, 'low_price__0': 8.938986485475738, 'stock__0': 'KIMBERA.MX', 'daily_change__0': -0.6055079680804345, 'daily_range__0': 1.082569125204136, 'type__0': 'Major Mexican', 'date__0': Timestamp('2009-12-04 00:00:00-0600', tz='America/Mexico_City'), 'open_price__0': 9.036816268297754, 'volume__0': 131

In [36]:
stock_df

,stock,date,open_price,high_price,low_price,close_price,volume,type,daily_change,daily_range,daily_return,volatility_20d,created_at
0,AAPL,1980-12-12 00:00:00-05:00,0.098485,0.098913,0.098485,0.098485,469033600,Big Tech,0.000000,0.434758,NaN,NaN,2025-08-21 13:54:24.823001
1,AAPL,1980-12-15 00:00:00-05:00,0.093775,0.093775,0.093347,0.093347,175884800,Big Tech,-0.456594,0.456594,-5.217026,NaN,2025-08-21 13:54:24.823001
2,AAPL,1980-12-16 00:00:00-05:00,0.086924,0.086924,0.086495,0.086495,105728000,Big Tech,-0.492577,0.492577,-7.339782,NaN,2025-08-21 13:54:24.823001
3,AAPL,1980-12-17 00:00:00-05:00,0.088636,0.089064,0.088636,0.088636,86441600,Big Tech,0.000000,0.483066,2.475033,NaN,2025-08-21 13:54:24.823001
4,AAPL,1980-12-18 00:00:00-05:00,0.091206,0.091634,0.091206,0.091206,73449600,Big Tech,0.000000,0.469449,2.899269,NaN,2025-08-21 13:54:24.823001
...,...,...,...,...,...,...,...,...,...,...,...,...,...
376894,GCARSOA1.MX,2025-08-15 00:00:00-06:00,128.979996,131.789993,128.899994,131.389999,85581,Major Mexican,1.868510,2.240657,2.296790,2.369952,2025-08-21 13:54:24.823001
376895,GCARSOA1.MX,2025-08-18 00:00:00-06:00,130.440002,130.440002,126.510002,128.589996,183418,Major Mexican,-1.418281,3.012880,-2.131063,2.365004,2025-08-21 13:54:24.823001
376896,GCARSOA1.MX,2025-08-19 00:00:00-06:00,127.790001,129.910004,127.669998,128.649994,98164,Major Mexican,0.672974,1.752880,0.046658,1.971483,2025-08-21 13:54:24.823001
376897,GCARSOA1.MX,2025-08-20 00:00:00-06:00,127.669998,128.039993,125.019997,125.870003,187987,Major Mexican,-1.409881,2.365471,-2.160895,1.582966,2025-08-21 13:54:24.823001


In [38]:
too_large_mask = (stock_df[["daily_change","daily_range","daily_return","volatility_20d","open_price","high_price","low_price","close_price"]].abs() >= 10000).any(axis=1)


In [42]:
print(stock_df[too_large_mask])

             stock                       date    open_price    high_price  \
208450       ^IXIC  2020-06-09 00:00:00-04:00   9867.190430  10002.500000   
208451       ^IXIC  2020-06-10 00:00:00-04:00  10012.320312  10086.889648   
208458       ^IXIC  2020-06-19 00:00:00-04:00  10042.129883  10053.910156   
208459       ^IXIC  2020-06-22 00:00:00-04:00   9945.490234  10059.610352   
208460       ^IXIC  2020-06-23 00:00:00-04:00  10130.830078  10221.849609   
...            ...                        ...           ...           ...   
360324  KIMBERA.MX  2011-03-24 00:00:00-06:00      0.000000     12.807361   
360325  KIMBERA.MX  2011-03-25 00:00:00-06:00      0.000000     12.736399   
360332  KIMBERA.MX  2011-04-05 00:00:00-05:00      0.000000     13.044335   
360354  KIMBERA.MX  2011-05-09 00:00:00-05:00      0.000000     12.238914   
360359  KIMBERA.MX  2011-05-16 00:00:00-05:00      0.000000     12.168874   

           low_price   close_price      volume           type daily_change 

In [43]:
# Replace INF values
stock_df.replace([np.inf, -np.inf], pd.NA, inplace=True)


In [44]:
## TIME TO UPLOAD

# Upload DataFrame
stock_df.to_sql(
    "historical_prices",
    con=engine,
    schema="public",
    if_exists="append",  # append to existing table
    index=False
)
print('done')

done


In [45]:
%reset

### IT DID WORK, but now it's time to test it back, loading ir from there

In [47]:
import yaml
import pandas as pd
from sqlalchemy import create_engine


# import yaml with credentials
with open("creds.yaml", "r") as file:
    creds = yaml.safe_load(file)["postgres"]
# Create connection
engine = create_engine(
    f"postgresql+psycopg2://{creds['user']}:{creds['password']}@{creds['host']}:{creds['port']}/{creds['database']}"
)

In [48]:
df = pd.read_sql("SELECT * FROM public.historical_prices", con=engine)

In [49]:
df.head()

,id,stock,date,open_price,high_price,low_price,close_price,volume,daily_change,daily_range,daily_return,volatility_20d,created_at,type
0,663376,AAPL,1980-12-11 23:00:00-06:00,0.0985,0.0989,0.0985,0.0985,469033600,0.0000,0.4348,NaN,NaN,2025-08-21 19:54:24.823001+00:00,Big Tech
1,663377,AAPL,1980-12-14 23:00:00-06:00,0.0938,0.0938,0.0933,0.0933,175884800,-0.4566,0.4566,-5.2170,NaN,2025-08-21 19:54:24.823001+00:00,Big Tech
2,663378,AAPL,1980-12-15 23:00:00-06:00,0.0869,0.0869,0.0865,0.0865,105728000,-0.4926,0.4926,-7.3398,NaN,2025-08-21 19:54:24.823001+00:00,Big Tech
3,663379,AAPL,1980-12-16 23:00:00-06:00,0.0886,0.0891,0.0886,0.0886,86441600,0.0000,0.4831,2.4750,NaN,2025-08-21 19:54:24.823001+00:00,Big Tech
4,663380,AAPL,1980-12-17 23:00:00-06:00,0.0912,0.0916,0.0912,0.0912,73449600,0.0000,0.4694,2.8993,NaN,2025-08-21 19:54:24.823001+00:00,Big Tech


In [50]:
df

,id,stock,date,open_price,high_price,low_price,close_price,volume,daily_change,daily_range,daily_return,volatility_20d,created_at,type
0,663376,AAPL,1980-12-11 23:00:00-06:00,0.0985,0.0989,0.0985,0.0985,469033600,0.0000,0.4348,NaN,NaN,2025-08-21 19:54:24.823001+00:00,Big Tech
1,663377,AAPL,1980-12-14 23:00:00-06:00,0.0938,0.0938,0.0933,0.0933,175884800,-0.4566,0.4566,-5.2170,NaN,2025-08-21 19:54:24.823001+00:00,Big Tech
2,663378,AAPL,1980-12-15 23:00:00-06:00,0.0869,0.0869,0.0865,0.0865,105728000,-0.4926,0.4926,-7.3398,NaN,2025-08-21 19:54:24.823001+00:00,Big Tech
3,663379,AAPL,1980-12-16 23:00:00-06:00,0.0886,0.0891,0.0886,0.0886,86441600,0.0000,0.4831,2.4750,NaN,2025-08-21 19:54:24.823001+00:00,Big Tech
4,663380,AAPL,1980-12-17 23:00:00-06:00,0.0912,0.0916,0.0912,0.0912,73449600,0.0000,0.4694,2.8993,NaN,2025-08-21 19:54:24.823001+00:00,Big Tech
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376894,1040270,GCARSOA1.MX,2025-08-15 00:00:00-06:00,128.9800,131.7900,128.9000,131.3900,85581,1.8685,2.2407,2.2968,2.3700,2025-08-21 19:54:24.823001+00:00,Major Mexican
376895,1040271,GCARSOA1.MX,2025-08-18 00:00:00-06:00,130.4400,130.4400,126.5100,128.5900,183418,-1.4183,3.0129,-2.1311,2.3650,2025-08-21 19:54:24.823001+00:00,Major Mexican
376896,1040272,GCARSOA1.MX,2025-08-19 00:00:00-06:00,127.7900,129.9100,127.6700,128.6500,98164,0.6730,1.7529,0.0467,1.9715,2025-08-21 19:54:24.823001+00:00,Major Mexican
376897,1040273,GCARSOA1.MX,2025-08-20 00:00:00-06:00,127.6700,128.0400,125.0200,125.8700,187987,-1.4099,2.3655,-2.1609,1.5830,2025-08-21 19:54:24.823001+00:00,Major Mexican


In [ ]:
## DONE! Pretty easy! now it's time to setup a routine. 